In [7]:
import ifcopenshell
import ifcopenshell.geom as geom
import numpy as np
import uuid

# --- Configuración de ifcopenshell.geom ---
# Necesario para el procesamiento geométrico
settings = geom.settings()

class IfcToOpenSeesConverter:
    def __init__(self, ifc_file_path):
        self.model = ifcopenshell.open(ifc_file_path)
        self.nodes_map = {}  # Mapea tuplas de coordenadas a IDs de nodos OpenSees
        self.next_node_id = 1
        self.opensees_commands = []
        self.node_tolerance = 0.01  # Tolerancia para agrupar nodos (en unidades del modelo IFC)

    def add_opensees_command(self, command):
        self.opensees_commands.append(command)

    def get_or_create_node(self, coords):
        """
        Obtiene un ID de nodo OpenSees para las coordenadas dadas.
        Si las coordenadas ya existen (dentro de la tolerancia), devuelve su ID.
        De lo contrario, crea un nuevo nodo y devuelve su ID.
        """
        # Redondea las coordenadas para la comparación de tolerancia
        rounded_coords = tuple(round(c, 5) for c in coords) # 5 decimales de precisión

        for existing_coords, node_id in self.nodes_map.items():
            if np.linalg.norm(np.array(rounded_coords) - np.array(existing_coords)) < self.node_tolerance:
                return node_id

        # Si no se encontró, crea un nuevo nodo
        node_id = self.next_node_id
        self.nodes_map[rounded_coords] = node_id
        self.next_node_id += 1
        self.add_opensees_command(f"ops.node({node_id}, {coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})")
        return node_id

    def extract_material_properties(self, ifc_element):
        """
        Intenta extraer propiedades del material asociadas al elemento.
        Esto es muy simplificado; en un caso real, necesitarías navegar por
        IfcRelAssociatesMaterial, IfcMaterialDefinitionSet, IfcMaterialProperties, etc.
        """
        # Por ahora, devuelve un material genérico.
        # Implementa aquí la lógica para buscar IfcMaterial o IfcMaterialProperties
        # y mapearlas a un material de OpenSees.
        # Puedes buscar en ifc_element.IsTypedBy.RelatedByType o ifc_element.IsDefinedBy.RelatedDefiners
        return {'E': 2.0e11, 'nu': 0.3, 'rho': 7850.0} # Acero por defecto

    def extract_section_properties(self, ifc_element):
        """
        Intenta extraer las propiedades de la sección transversal.
        Muy simplificado; necesitas buscar IfcRelDefinesByProperties, IfcSectionProperties,
        IfcProfileDef (IfcRectangleProfileDef, IfcIProfileDef, etc.)
        """
        # Por ahora, devuelve propiedades genéricas.
        # Implementa aquí la lógica para buscar IfcProfileDef y calcular A, Ixx, Iyy, J.
        # Ejemplo: Para una viga rectangular de 0.3x0.6
        if 'Beam' in ifc_element.is_a() or 'Column' in ifc_element.is_a():
             # Intenta buscar un perfil. Esto es un placeholder.
            # Necesitarías recorrer ifc_element.IsTypedBy, etc.
            return {'A': 0.3 * 0.6, 'Ixx': (0.3 * 0.6**3) / 12, 'Iyy': (0.6 * 0.3**3) / 12, 'G': None, 'J': None}
        return {'A': 1.0, 'Ixx': 1.0, 'Iyy': 1.0, 'G': None, 'J': None} # Valores por defecto

    def process_beams_and_columns(self):
        linear_elements = self.model.by_type('IfcBeam') + self.model.by_type('IfcColumn')
        frame_elements_count = 0
        self.add_opensees_command("\n# --- Definicion de Materiales OpenSees (simplificado) ---")
        self.add_opensees_command("ops.uniaxialMaterial('Elastic', 1, 2.0e11) # ID 1 para material generico") # Ajustar esto

        self.add_opensees_command("\n# --- Definicion de Secciones (transformaciones de seccion) ---")
        # Ejemplo de transformación de sección para elementos frame
        self.add_opensees_command("ops.geomTransf('Linear', 1) # ID 1 para transformacion generica")

        self.add_opensees_command("\n# --- Elementos Frame (Vigas y Columnas) ---")

        for i, element in enumerate(linear_elements):
            try:
                # Intenta obtener la línea central del elemento
                # Esto es una simplificación. ifcopenshell.geom.tessellate es para mallas,
                # para líneas centrales se necesita más procesamiento del IfcProductDefinitionShape.
                # Una forma común es buscar IfcProduct.Representation.Representations[0].Item[0]
                # que podría ser un IfcPolyline o IfcArbitraryOpenProfileDef.
                
                # Por simplicidad, intentaremos obtener la bounding box y usar su centro
                # Esto NO es una línea de eje precisa, solo una aproximación para el ejemplo.
                # Para un control preciso, necesitas analizar IfcAxis2Placement3D
                # y las curvas de extrusión o las representaciones de línea.
                
                # ifcopenshell.geom.utils no tiene una forma directa de obtener lineas de eje
                # Esto requeriría una implementación más profunda de la geometría
                
                # Una alternativa es usar las propiedades de la Pset_BeamCommon o Pset_ColumnCommon
                # si existen, para encontrar puntos de inicio/fin, pero no es universal.

                # Fallback: Usar los puntos de inicio/fin de la línea de extrusión si disponible
                # O la línea del IfcCurve geométrica asociada
                if element.ObjectPlacement and element.ObjectPlacement.RelativePlacement:
                    # Intenta extraer el origen y la dirección principal
                    placement = element.ObjectPlacement.RelativePlacement
                    origin = placement.Location.Coordinates
                    
                    # Para una viga/columna, a menudo son extrusiones.
                    # Asumimos que la viga/columna se extiende a lo largo de un eje.
                    # Esto es MUY simplificado.
                    # Un enfoque más robusto implicaría usar el IfcGeometricRepresentationItem
                    # y analizar IfcProductDefinitionShape para IfcExtrudedAreaSolid
                    # y sacar el Axis de la extrusión.
                    
                    # Placeholder para start_point y end_point
                    # Ejemplo: Usar los extremos de la bounding box si no hay otra forma
                    product_shape = geom.create_shape(settings, element)
                    bbox = product_shape.bbox() # [x_min, y_min, z_min, x_max, y_max, z_max]

                    start_point = np.array([bbox[0], bbox[1], bbox[2]]) # esquina inf-izq-frontal
                    end_point = np.array([bbox[3], bbox[4], bbox[5]])   # esquina sup-der-trasera

                    # Para elementos lineales, intentamos usar el centro de dos caras opuestas o el centro de la extrusión
                    # Esto es un gran desafío en IFC sin reglas claras de Revit.
                    # Mejor aproximación: intentar usar la línea de extrusión si el elemento es IfcExtrudedAreaSolid
                    
                    if element.Representation and element.Representation.Representations:
                        for rep in element.Representation.Representations:
                            for item in rep.Items:
                                if item.is_a('IfcExtrudedAreaSolid'):
                                    extrusion_axis = item.ExtrudedDirection.DirectionRatios
                                    extrusion_length = item.Depth
                                    # Esto es el origen del perfil de la extrusión, no necesariamente el centro de la viga
                                    # start_point_geom = item.SweptArea.RefDirection.Location.Coordinates 
                                    # end_point_geom = start_point_geom + np.array(extrusion_axis) * extrusion_length
                                    
                                    # Para este ejemplo, seguiremos con una aproximación de bounding box
                                    # o mejor aún, si tuvieras los puntos de inicio/fin del modelo analítico de Revit
                                    pass # Mantener los puntos del bounding box para este ejemplo

                    # Si el elemento tiene un IfcCurve, se puede extraer directamente
                    # Esto es lo ideal para la línea analítica
                    # Por ejemplo, si el elemento IfcBeam está asociado a un IfcStructuralCurveMember
                    # que tiene un IfcCurve.

                    # Para este ejemplo, usaremos el centro de las caras opuestas de la bounding box para los puntos del elemento 'frame'.
                    # Esto no es robusto para un análisis estructural preciso, pero sirve para la demostración.
                    
                    # Obtener el centro del bounding box
                    center_x = (bbox[0] + bbox[3]) / 2
                    center_y = (bbox[1] + bbox[4]) / 2
                    center_z = (bbox[2] + bbox[5]) / 2

                    # Suponemos que el elemento se extiende predominantemente en X o Y o Z
                    # Vamos a simplificar y decir que el elemento va de un punto a otro.
                    # La forma más simple es usar los extremos de la línea diagonal de la bounding box
                    # o los centros de las caras opuestas.
                    
                    # Un enfoque más realista para la línea de eje:
                    # Intenta encontrar el IfcAxis2Placement3D y la dirección de extrusión
                    # Esto es un placeholder; la extracción real es más compleja.
                    
                    # Asumiremos puntos de inicio y fin dummy para el ejemplo que demuestren el flujo
                    # En la vida real, los obtendrías de la geometría extrudida o la línea del modelo analítico
                    
                    start_point = np.array([bbox[0], (bbox[1]+bbox[4])/2, (bbox[2]+bbox[5])/2])
                    end_point = np.array([bbox[3], (bbox[1]+bbox[4])/2, (bbox[2]+bbox[5])/2])
                    
                    # Si la viga/columna es más alta que ancha, podría estar en Z
                    dim_x = bbox[3] - bbox[0]
                    dim_y = bbox[4] - bbox[1]
                    dim_z = bbox[5] - bbox[2]

                    if dim_z > dim_x and dim_z > dim_y: # Predominantemente vertical (columna)
                         start_point = np.array([(bbox[0]+bbox[3])/2, (bbox[1]+bbox[4])/2, bbox[2]])
                         end_point = np.array([(bbox[0]+bbox[3])/2, (bbox[1]+bbox[4])/2, bbox[5]])
                    elif dim_x > dim_y and dim_x > dim_z: # Predominantemente horizontal en X (viga)
                         start_point = np.array([bbox[0], (bbox[1]+bbox[4])/2, (bbox[2]+bbox[5])/2])
                         end_point = np.array([bbox[3], (bbox[1]+bbox[4])/2, (bbox[2]+bbox[5])/2])
                    else: # Predominantemente horizontal en Y (viga)
                         start_point = np.array([(bbox[0]+bbox[3])/2, bbox[1], (bbox[2]+bbox[5])/2])
                         end_point = np.array([(bbox[0]+bbox[3])/2, bbox[4], (bbox[2]+bbox[5])/2])


                    node_id_start = self.get_or_create_node(start_point)
                    node_id_end = self.get_or_create_node(end_point)

                    material_props = self.extract_material_properties(element)
                    section_props = self.extract_section_properties(element)

                    # ID de elemento único para OpenSees
                    element_id = int(uuid.uuid4().int % 1000000) # Genera un ID "aleatorio"

                    # Definición de elemento 'elasticBeamColumn' en OpenSeesPy
                    # Los valores de propiedades A, E, G, J, Ixx, Iyy deben venir de section_props y material_props
                    # G = E / (2 * (1 + nu))
                    G_val = material_props['E'] / (2 * (1 + material_props['nu'])) if material_props['nu'] is not None else material_props['E'] / 2.6 # Estimación si nu es nulo
                    
                    self.add_opensees_command(
                        f"ops.element('elasticBeamColumn', {element_id}, "
                        f"{node_id_start}, {node_id_end}, "
                        f"{section_props['A']:.5f}, {material_props['E']:.0f}, "
                        f"{G_val:.0f}, {section_props['J'] if section_props['J'] else 0.0:.5f}, "
                        f"{section_props['Iyy']:.5f}, {section_props['Ixx']:.5f}, "
                        f"1) # Ultimo 1 es el tag de la transformacion geometrica"
                    )
                    frame_elements_count += 1

            except Exception as e:
                print(f"Advertencia: No se pudo procesar el elemento IfcBeam/IfcColumn '{element.Name}' (GlobalId: {element.GlobalId}). Error: {e}")
                # Puedes loguear el error o saltar este elemento

        print(f"Procesados {frame_elements_count} elementos 'frame' (vigas/columnas).")


    def process_walls_and_slabs(self):
        surface_elements = self.model.by_type('IfcWall') + self.model.by_type('IfcSlab')
        shell_elements_count = 0
        self.add_opensees_command("\n# --- Elementos Shell (Muros y Losas) ---")

        for i, element in enumerate(surface_elements):
            try:
                # La extracción de la geometría de una superficie (plano medio, vértices)
                # es más compleja. Ifcopenshell.geom.tessellate devuelve una malla (triángulos).
                # Necesitamos extraer los vértices del contorno o el plano medio para un elemento 'shell'.
                
                # Una estrategia es obtener la malla y usar los vértices de la cara principal.
                # Esto es una simplificación: asumiremos que cada elemento es una única superficie plana.
                
                # ifcopenshell.util.selector.get_representation es útil para obtener la geometría principal
                # pero el procesamiento para obtener 4 vértices para un shell plano requiere análisis.
                
                product_shape = geom.create_shape(settings, element)
                
                if not product_shape.geometry:
                    print(f"Advertencia: No hay geometría disponible para {element.Name} (GlobalId: {element.GlobalId}).")
                    continue

                # Intenta obtener los vértices del contorno principal de la superficie.
                # Esto es MUY simplificado. Una pared o losa puede tener aberturas o formas complejas.
                # Aquí, solo tomaremos los vértices de la malla más externa o principal.
                
                # Obtener los vértices de la malla (triangulación)
                # Esto es útil para visualización, no directamente para elementos shell de OpenSees.
                # Para un shell, necesitamos nodos en las esquinas.

                # Si el elemento es un IfcPlate o IfcWall, el espesor puede ser una propiedad
                # o derivado de la geometría.
                thickness = None
                # Intenta buscar una propiedad de espesor
                for rel_def in element.IsDefinedBy:
                    if rel_def.is_a('IfcRelDefinesByProperties'):
                        prop_set = rel_def.RelatingPropertyDefinition
                        if prop_set.is_a('IfcPropertySet'):
                            for prop in prop_set.HasProperties:
                                if prop.is_a('IfcPropertySingleValue') and prop.Name.lower() == 'thickness' and prop.NominalValue:
                                    thickness = prop.NominalValue.wrappedValue
                                    break
                        elif prop_set.is_a('IfcElementQuantity'):
                             for quantity in prop_set.Quantities:
                                 if quantity.is_a('IfcQuantityLength') and quantity.Name.lower() == 'thickness':
                                     thickness = quantity.LengthValue
                                     break
                    if thickness is not None:
                        break
                
                if thickness is None:
                    # Si no se encuentra, intentar inferir del bounding box
                    bbox = product_shape.bbox()
                    dims = [bbox[3]-bbox[0], bbox[4]-bbox[1], bbox[5]-bbox[2]]
                    # El espesor será la dimensión más pequeña
                    thickness = min(dims)
                    if thickness < 0.001: # Si es demasiado delgado, asumir un valor por defecto o saltar
                        thickness = 0.2 # Valor por defecto para paredes/losas si no se encuentra
                        print(f"Advertencia: Espesor no encontrado para {element.Name}. Usando valor por defecto: {thickness:.3f}m")

                
                # Para un elemento Shell en OpenSees, necesitamos 4 nodos que definan un plano.
                # Esto es difícil de generalizar desde IFC.
                # Una aproximación: Obtener el centroide del bounding box y los puntos extremos de la cara principal.
                
                bbox = product_shape.bbox()
                
                # Asumimos que la cara principal es XY para losas o XZ/YZ para muros
                # Genera 4 puntos representativos de la cara principal
                # Esto es una HEURÍSTICA y podría no funcionar para todas las geometrías.
                
                # Muros (predominantemente verticales)
                if 'Wall' in element.is_a():
                    # Suponemos que un muro se extiende en X y Z (o Y y Z)
                    # Y tiene un espesor en Y (o X)
                    p1 = np.array([bbox[0], (bbox[1]+bbox[4])/2, bbox[2]])
                    p2 = np.array([bbox[3], (bbox[1]+bbox[4])/2, bbox[2]])
                    p3 = np.array([bbox[3], (bbox[1]+bbox[4])/2, bbox[5]])
                    p4 = np.array([bbox[0], (bbox[1]+bbox[4])/2, bbox[5]])
                # Losas (predominantemente horizontales)
                elif 'Slab' in element.is_a():
                    # Suponemos que una losa se extiende en X y Y
                    # Y tiene un espesor en Z
                    p1 = np.array([bbox[0], bbox[1], (bbox[2]+bbox[5])/2])
                    p2 = np.array([bbox[3], bbox[1], (bbox[2]+bbox[5])/2])
                    p3 = np.array([bbox[3], bbox[4], (bbox[2]+bbox[5])/2])
                    p4 = np.array([bbox[0], bbox[4], (bbox[2]+bbox[5])/2])
                else:
                    continue # No se sabe cómo manejar este tipo de elemento de superficie

                node_id1 = self.get_or_create_node(p1)
                node_id2 = self.get_or_create_node(p2)
                node_id3 = self.get_or_create_node(p3)
                node_id4 = self.get_or_create_node(p4)

                material_props = self.extract_material_properties(element)
                
                element_id = int(uuid.uuid4().int % 1000000) # Genera un ID "aleatorio"

                # Definición de material para shell (usaremos el mismo ID 1 por simplicidad)
                # En un caso real, definirías materiales específicos como PlateFiber o Section
                
                # OpenSees necesita un material para el shell
                # Usaremos un material de hormigón genérico para el ejemplo
                # self.add_opensees_command(f"ops.nDMaterial('ElasticIsotropic', 2, {material_props['E']:.0f}, {material_props['nu']:.2f})") # ID 2 para shell
                # self.add_opensees_command(f"ops.section('PlateFiber', {element_id}, 2, {thickness:.3f})") # Sección delgada de placa, ID de sección es el mismo que el elemento

                self.add_opensees_command(
                    f"ops.element('ShellDKGQ', {element_id}, "
                    f"{node_id1}, {node_id2}, {node_id3}, {node_id4}, "
                    f"1, {thickness:.3f}) # ID 1 es el material para el shell (simplificado)"
                )
                shell_elements_count += 1

            except Exception as e:
                print(f"Advertencia: No se pudo procesar el elemento IfcWall/IfcSlab '{element.Name}' (GlobalId: {element.GlobalId}). Error: {e}")
        
        print(f"Procesados {shell_elements_count} elementos 'shell' (muros/losas).")


    def generate_opensees_script(self, output_file="model_opensees.py"):
        """
        Genera el script de OpenSeesPy a partir de los comandos recolectados.
        """
        with open(output_file, 'w') as f:
            f.write("import openseespy.opensees as ops\n")
            f.write("import numpy as np\n\n")
            f.write("ops.wipe()\n")
            f.write("ops.model('3D', '-ndm', 3, '-ndf', 6)\n\n")
            
            f.write("# --- Definición de Nodos ---\n")
            # Los nodos ya se agregan cuando se crean para evitar duplicados
            # Ahora solo los escribimos en el orden en que fueron creados
            sorted_nodes = sorted(self.nodes_map.items(), key=lambda item: item[1])
            for coords, node_id in sorted_nodes:
                f.write(f"ops.node({node_id}, {coords[0]:.3f}, {coords[1]:.3f}, {coords[2]:.3f})\n")
            
            # Escribir el resto de comandos (materiales, transformaciones, elementos)
            for command in self.opensees_commands:
                f.write(command + "\n")
            
            f.write("\n# --- Definición de Apoyos (Ejemplo: empotramiento en la base) ---\n")
            f.write("# Esto es un ejemplo. Necesitarás identificar los nodos base de tus columnas.\n")
            f.write("# for node_id, coords in converter.nodes_map.items():\n")
            f.write("#     if coords[2] < 0.05: # Si el nodo está cerca del nivel z=0 (asumido como base)\n")
            f.write("#         ops.fix(node_id, 1, 1, 1, 1, 1, 1) # Empotrado\n")
            
            f.write("\n# --- Definición de Cargas (Ejemplo: gravedad) ---\n")
            f.write("# ops.timeSeries('Linear', 1)\n")
            f.write("# ops.pattern('Plain', 1, 1)\n")
            f.write("# for node_id, coords in converter.nodes_map.items():\n")
            f.write("#     if 'columna' in str(node_id): # Necesitas una forma de identificar los nodos de columna\n")
            f.write("#         ops.load(node_id, 0.0, 0.0, -10.0, 0.0, 0.0, 0.0) # Carga axial en columnas\n")

            f.write("\n# --- Análisis (Ejemplo: estático) ---\n")
            f.write("# ops.constraints('Plain')\n")
            f.write("# ops.numberer('RCM')\n")
            f.write("# ops.system('BandSPD')\n")
            f.write("# ops.integrator('LoadControl', 0.1)\n")
            f.write("# ops.algorithm('Newton')\n")
            f.write("# ops.analysis('Static')\n")
            f.write("# ops.analyze(10) # 10 pasos de carga\n")
            f.write("# ops.printModel('-JSON', 'model.json')\n")

        print(f"\nScript de OpenSeesPy generado en '{output_file}'")


# --- Uso del Convertidor ---
if __name__ == "__main__":
    ifc_file = "sample_models/CASA TIPO 11111.ifc" # ¡CAMBIA ESTO A LA RUTA DE TU ARCHIVO IFC!

    try:
        converter = IfcToOpenSeesConverter(ifc_file)
        
        print("\nIniciando procesamiento de vigas y columnas...")
        converter.process_beams_and_columns()
        
        print("\nIniciando procesamiento de muros y losas...")
        converter.process_walls_and_slabs()
        
        converter.generate_opensees_script()
        print("\nProceso de conversión completado.")

    except Exception as e:
        print(f"\n¡Ocurrió un error general! Asegúrate de que el archivo IFC exista y sea válido.")
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()


Iniciando procesamiento de vigas y columnas...
Advertencia: No se pudo procesar el elemento IfcBeam/IfcColumn 'M_Concrete-Rectangular Beam:250 x 500mm:416913' (GlobalId: 1YMpECnoXAAgd3TsblgkRb). Error: 'TriangulationElement' object has no attribute 'bbox'
Advertencia: No se pudo procesar el elemento IfcBeam/IfcColumn 'M_Concrete-Rectangular Beam:250 x 500mm:416923' (GlobalId: 1YMpECnoXAAgd3TsblgkRl). Error: 'TriangulationElement' object has no attribute 'bbox'
Advertencia: No se pudo procesar el elemento IfcBeam/IfcColumn 'M_Concrete-Rectangular Beam:250 x 500mm:418561' (GlobalId: 2e3KIcTBT93wdBtSLDrMFt). Error: 'TriangulationElement' object has no attribute 'bbox'
Advertencia: No se pudo procesar el elemento IfcBeam/IfcColumn 'M_Concrete-Rectangular Beam:250 x 500mm:418605' (GlobalId: 2e3KIcTBT93wdBtSLDrMFR). Error: 'TriangulationElement' object has no attribute 'bbox'
Advertencia: No se pudo procesar el elemento IfcBeam/IfcColumn 'M_Concrete-Rectangular Beam:250 x 500mm:418641' (Glo

In [ ]:
import clr
clr.AddReference("RevitAPI")
clr.AddReference("RevitAPIUI")
from Autodesk.Revit.DB import *
from Autodesk.Revit.DB.Structure import *
from Autodesk.Revit.UI import *
import RevitServices
from RevitServices.Persistence import DocumentManager
from RevitServices.Transactions import TransactionManager
import json
import sys

# Obtener el documento actual
doc = DocumentManager.Instance.CurrentDBDocument

# Inicializar diccionario para datos
analytical_data = {
    "nodes": [],
    "frames": [],
    "shells": []
}

# Diccionario para nodos únicos
node_dict = {}  # Clave: tupla (x,y,z), Valor: info del nodo
node_counter = 0

# --- Función para obtener propiedades del elemento ---
def get_element_properties(element):
    props = {}
    if not element:
        return props
    
    # Propiedades básicas
    props["Name"] = element.Name if hasattr(element, 'Name') else "N/A"
    props["Category"] = element.Category.Name if element.Category else "N/A"
    
    # Parámetros del elemento
    for param in element.GetOrderedParameters():
        if param.HasValue:
            try:
                if param.StorageType == StorageType.Double:
                    props[param.Definition.Name] = param.AsDouble()
                elif param.StorageType == StorageType.Integer:
                    props[param.Definition.Name] = param.AsInteger()
                elif param.StorageType == StorageType.String:
                    props[param.Definition.Name] = param.AsString()
                elif param.StorageType == StorageType.ElementId:
                    props[param.Definition.Name] = param.AsElementId().IntegerValue
            except:
                continue
    
    # Propiedades del tipo si existe
    if hasattr(element, 'GetTypeId'):
        type_id = element.GetTypeId()
        if type_id != ElementId.InvalidElementId:
            type_element = doc.GetElement(type_id)
            if type_element:
                props["Type Name"] = type_element.Name
                # Materiales
                if hasattr(type_element, 'GetMaterialIds'):
                    material_ids = type_element.GetMaterialIds(False)
                    if material_ids and len(material_ids) > 0:
                        material = doc.GetElement(material_ids[0])
                        if material:
                            props["Material"] = material.Name
                            # Propiedades del material
                            for mat_param in material.GetOrderedParameters():
                                if mat_param.HasValue:
                                    try:
                                        props[f"Material_{mat_param.Definition.Name}"] = mat_param.AsDouble() if mat_param.StorageType == StorageType.Double else mat_param.AsString()
                                    except:
                                        continue
    return props

# --- Función para registrar nodos ---
def register_node(point):
    global node_counter
    node_key = (round(point.X, 8), round(point.Y, 8), round(point.Z, 8))
    if node_key not in node_dict:
        node_dict[node_key] = {
            "id": node_counter,
            "x": point.X,
            "y": point.Y,
            "z": point.Z,
            "release_conditions": []
        }
        analytical_data["nodes"].append(node_dict[node_key])
        node_counter += 1
    return node_dict[node_key]["id"]

# --- Recolectar Elementos Analíticos ---
try:
    element_collector = FilteredElementCollector(doc).OfClass(AnalyticalModel).WhereElementIsNotElementType().ToElements()
    
    for elem in element_collector:
        if not elem.IsActive:
            continue

        # Información básica del elemento
        element_info = {
            "id": elem.Id.IntegerValue,
            "type": None,
            "physical_info": None
        }

        # Obtener elemento físico asociado
        try:
            association = elem.GetAnalyticalToPhysicalAssociation()
            physical_element = doc.GetElement(association.GetAssociatedPhysicalElementId()) if association else None
            if physical_element:
                element_info["physical_info"] = {
                    "id": physical_element.Id.IntegerValue,
                    "properties": get_element_properties(physical_element)
                }
        except Exception as e:
            print(f"Error obteniendo elemento físico para {elem.Id}: {str(e)}")
            element_info["physical_info"] = None

        # Procesar según el tipo de elemento analítico
        try:
            if isinstance(elem, AnalyticalModelFrame):
                curve = elem.GetCurve()
                if curve:
                    start_id = register_node(curve.GetEndPoint(0))
                    end_id = register_node(curve.GetEndPoint(1))
                    
                    element_info.update({
                        "type": "frame",
                        "start_node_id": start_id,
                        "end_node_id": end_id,
                        "length": curve.Length,
                        "curve_type": str(curve.GetType().Name)
                    })
                    
                    # Liberaciones
                    try:
                        releases = elem.GetAnalyticalRelease()
                        if releases:
                            element_info["start_release"] = str(releases.Start)
                            element_info["end_release"] = str(releases.End)
                    except:
                        pass
                    
                    analytical_data["frames"].append(element_info)
            
            elif isinstance(elem, AnalyticalModelSurface):
                boundary_loops = elem.GetBoundaryLoops()
                if boundary_loops:
                    loops_info = []
                    for loop in boundary_loops:
                        loop_nodes = []
                        for curve in loop:
                            point = curve.GetEndPoint(0)
                            node_id = register_node(point)
                            loop_nodes.append(node_id)
                        loops_info.append(loop_nodes)
                    
                    element_info.update({
                        "type": "shell",
                        "loops": loops_info
                    })
                    
                    analytical_data["shells"].append(element_info)
        except Exception as e:
            print(f"Error procesando elemento {elem.Id}: {str(e)}")
            continue

except Exception as e:
    print(f"Error en la recolección de elementos: {str(e)}")
    OUT = str(e)
    sys.exit()

# --- Salida ---
try:
    output_json = json.dumps(analytical_data, indent=4)
    OUT = output_json
except Exception as e:
    OUT = f"Error al generar JSON: {str(e)}"